In [ ]:
# Pandas Approach for Olgas H505 usecase ...

In [49]:
gap_26 = pd.read_parquet('gap_from_opdb_since_january_2021_26.parquet')
gap_27 = pd.read_parquet('gap_from_opdb_since_january_2021_27.parquet')

In [55]:
gap_26 = gap_26.dropna(subset=["Batchnumber"])
df_26=gap_26[gap_26["Batchnumber"].str.startswith("10000411")]

gap_27 = gap_27.dropna(subset=["Batchnumber"])
df_27=gap_27[gap_27["Batchnumber"].str.startswith("10000411")]

In [59]:
# df_26 # 37 rows × 102 columns
# df_27 # 37 rows × 102 columns

In [2]:
# Reference: https://stackoverflow.com/questions/43593554/merging-two-dataframes-based-on-a-date-between-two-other-dates-without-a-common

import pandas as pd

l4data = pd.DataFrame({'start_date':['2017-03-27 09:40:10','2017-01-10 22:10:11'],'end_date':['2017-04-20 10:30:22','2017-02-01 20:01:01']})
pimsdata = pd.DataFrame({'event_date':['2017-01-20 23:01:00','2017-01-27 09:40:10'],'value':[100,200]})
tags_uniqueids = pd.DataFrame({'event_date':['2017-01-20 23:01:00','2017-01-27 09:40:10'],'tags':["abc","def"]})

l4data['end_date'] = pd.to_datetime(l4data.end_date)
l4data['start_date'] = pd.to_datetime(l4data.start_date)
pimsdata['event_date'] = pd.to_datetime(pimsdata.event_date)
tags_uniqueids['event_date'] = pd.to_datetime(pimsdata.event_date)

In [3]:
tags_uniqueids.head()

,event_date,tags
0,2017-01-20 23:01:00,abc
1,2017-01-27 09:40:10,def


In [2]:
pimsdata.head()

,event_date,value
0,2017-01-20 23:01:00,100
1,2017-01-27 09:40:10,200


In [6]:
from pandasql import sqldf

## TODO time format changed, no idea why?
def merge(pimsdata, l4data,uniqueIds):
    cond_join= """
        SELECT 
            pimsdata.*,
            l4data.*,
            uniqueIds.tags
        FROM pimsdata
        JOIN l4data
        ON pimsdata.[event_date] >= l4data.[start_date] 
        AND pimsdata.[event_date] <= l4data.[end_date]
        JOIN uniqueIds
        ON pimsdata.[event_date] == uniqueIds.[event_date]         
    """
    return sqldf(cond_join)

In [7]:
merge(pimsdata, l4data, tags_uniqueids)

,event_date,value,start_date,end_date,tags
0,2017-01-20 23:01:00.000000,100,2017-01-10 22:10:11.000000,2017-02-01 20:01:01.000000,abc
1,2017-01-27 09:40:10.000000,200,2017-01-10 22:10:11.000000,2017-02-01 20:01:01.000000,def


In [4]:
# this cross operations is also may cause performance issue tbh!!
df_merge = pimsdata.merge(l4data, how='cross')

In [5]:
df_merge.head()

,event_date,value,start_date,end_date
0,2017-01-20 23:01:00,100,2017-03-27 09:40:10,2017-04-20 10:30:22
1,2017-01-20 23:01:00,100,2017-01-10 22:10:11,2017-02-01 20:01:01
2,2017-01-27 09:40:10,200,2017-03-27 09:40:10,2017-04-20 10:30:22
3,2017-01-27 09:40:10,200,2017-01-10 22:10:11,2017-02-01 20:01:01


In [6]:
df_merge = df_merge.query('event_date >= start_date and event_date <= end_date')

In [7]:
df_merge

,event_date,value,start_date,end_date
1,2017-01-20 23:01:00,100,2017-01-10 22:10:11,2017-02-01 20:01:01
3,2017-01-27 09:40:10,200,2017-01-10 22:10:11,2017-02-01 20:01:01


In [20]:
old_df=pd.read_parquet("10000411_old.parquet")
old_df # 56709 rows × 10 columns

,UniqueId,Value,TimeStamp,AssetUnitIdentifier,Materialnumber,Batchnumber,SegmentStartTime,SegmentEndTime,SegmentStartTime_formatted,SegmentEndTime_formatted
0,5016523955535,56.79000,2022-04-24 11:28:00.589000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000
1,5016523955535,56.86200,2022-04-24 11:28:01.393000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000
2,5016523963704,1.27200,2022-04-24 11:28:02.003000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000
3,5016523970251,8713.00000,2022-04-24 11:28:03.980000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000
4,5016523955535,56.86200,2022-04-24 11:28:04.620000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000
...,...,...,...,...,...,...,...,...,...,...
56704,5016523976266,91.96655,2022-04-24 16:16:58.659000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000
56705,5016523978810,-38.35286,2022-04-24 16:16:58.659000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000
56706,5016523955535,96.79300,2022-04-24 16:16:58.953000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000
56707,5016523952192,2715.84000,2022-04-24 16:16:59.021000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000


In [31]:
old_ts = old_df["TimeStamp"]

In [60]:
old_ts_min = old_df['TimeStamp'].min()
old_ts_max = old_df['TimeStamp'].max()

print(old_ts_min, old_ts_max)

2022-04-24 11:28:00.589000 2022-04-24 16:16:59.318000


In [61]:
old_ts # 56709

0        2022-04-24 11:28:00.589000
1        2022-04-24 11:28:01.393000
2        2022-04-24 11:28:02.003000
3        2022-04-24 11:28:03.980000
4        2022-04-24 11:28:04.620000
                    ...            
56704    2022-04-24 16:16:58.659000
56705    2022-04-24 16:16:58.659000
56706    2022-04-24 16:16:58.953000
56707    2022-04-24 16:16:59.021000
56708    2022-04-24 16:16:59.318000
Name: TimeStamp, Length: 56709, dtype: object

In [63]:
# 2022-04-24 11:28:00.589000 2022-04-24 16:16:59.318000

new_df[(new_df.TimeStamp >= old_ts_min) & (new_df.TimeStamp <= old_ts_max)] # 56709

,UniqueId,Value,TimeStamp,AssetUnitIdentifier,Materialnumber,Batchnumber,SegmentStartTime,SegmentEndTime,SegmentStartTime_formatted,SegmentEndTime_formatted,BackendTag
246441,5016523955535,56.790000,2022-04-24 11:28:00.589000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000,F501097_MASS
246442,5016523955535,56.862000,2022-04-24 11:28:01.393000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000,F501097_MASS
246443,5016523963704,1.272000,2022-04-24 11:28:02.003000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000,P501081
246444,5016523970251,8713.000000,2022-04-24 11:28:03.980000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000,L501000
246445,5016523955535,56.862000,2022-04-24 11:28:04.620000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000,F501097_MASS
...,...,...,...,...,...,...,...,...,...,...,...
303145,5016523967137,29758.400000,2022-04-24 16:16:58.659000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000,U506000
303146,5016523956271,30.186604,2022-04-24 16:16:58.659000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000,U506001H2
303147,5016523955535,96.793000,2022-04-24 16:16:58.953000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000,F501097_MASS
303148,5016523952192,2715.840000,2022-04-24 16:16:59.021000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000,F501040C


In [32]:
new_df[~new_df.TimeStamp.isin(old_ts)]

,UniqueId,Value,TimeStamp,AssetUnitIdentifier,Materialnumber,Batchnumber,SegmentStartTime,SegmentEndTime,SegmentStartTime_formatted,SegmentEndTime_formatted,BackendTag
0,5016523970349,29.136250,2022-03-05 04:39:02.294000,R5010,10000411,896,2022-03-05 04:39:00.0000000 +01:00,2022-03-05 11:41:00.0000000 +01:00,2022-03-05 04:39:00.000000,2022-03-05 11:41:00.000000,S501000
1,5016523955926,277.568756,2022-03-05 04:39:03.760000,R5010,10000411,896,2022-03-05 04:39:00.0000000 +01:00,2022-03-05 11:41:00.0000000 +01:00,2022-03-05 04:39:00.000000,2022-03-05 11:41:00.000000,F501002
2,5016523961394,95.936600,2022-03-05 04:39:04.151000,R5010,10000411,896,2022-03-05 04:39:00.0000000 +01:00,2022-03-05 11:41:00.0000000 +01:00,2022-03-05 04:39:00.000000,2022-03-05 11:41:00.000000,H506003C
3,5016523980111,890.880000,2022-03-05 04:39:04.913000,R5010,10000411,896,2022-03-05 04:39:00.0000000 +01:00,2022-03-05 11:41:00.0000000 +01:00,2022-03-05 04:39:00.000000,2022-03-05 11:41:00.000000,P501000
4,5016523978452,0.705643,2022-03-05 04:39:05.000000,R5010,10000411,896,2022-03-05 04:39:00.0000000 +01:00,2022-03-05 11:41:00.0000000 +01:00,2022-03-05 04:39:00.000000,2022-03-05 11:41:00.000000,U501002H2
...,...,...,...,...,...,...,...,...,...,...,...
246436,5016523955865,4645.051270,2022-04-20 11:57:59.654000,R5010,10000411,902,2022-04-20 07:12:00.0000000 +02:00,2022-04-20 11:58:00.0000000 +02:00,2022-04-20 07:12:00.000000,2022-04-20 11:58:00.000000,F501090
246437,5016523961332,188.000000,2022-04-20 11:57:59.654000,R5010,10000411,902,2022-04-20 07:12:00.0000000 +02:00,2022-04-20 11:58:00.0000000 +02:00,2022-04-20 07:12:00.000000,2022-04-20 11:58:00.000000,F501061C
246438,5016523961346,6848.000490,2022-04-20 11:57:59.654000,R5010,10000411,902,2022-04-20 07:12:00.0000000 +02:00,2022-04-20 11:58:00.0000000 +02:00,2022-04-20 07:12:00.000000,2022-04-20 11:58:00.000000,F501090F
246439,5016523970626,364.000000,2022-04-20 11:57:59.654000,R5010,10000411,902,2022-04-20 07:12:00.0000000 +02:00,2022-04-20 11:58:00.0000000 +02:00,2022-04-20 07:12:00.000000,2022-04-20 11:58:00.000000,F501052C


In [21]:
new_df=pd.read_parquet("10000411_new.parquet")
new_df # 303125 rows rows × 11 columns

,UniqueId,Value,TimeStamp,AssetUnitIdentifier,Materialnumber,Batchnumber,SegmentStartTime,SegmentEndTime,SegmentStartTime_formatted,SegmentEndTime_formatted,BackendTag
0,5016523970349,29.136250,2022-03-05 04:39:02.294000,R5010,10000411,896,2022-03-05 04:39:00.0000000 +01:00,2022-03-05 11:41:00.0000000 +01:00,2022-03-05 04:39:00.000000,2022-03-05 11:41:00.000000,S501000
1,5016523955926,277.568756,2022-03-05 04:39:03.760000,R5010,10000411,896,2022-03-05 04:39:00.0000000 +01:00,2022-03-05 11:41:00.0000000 +01:00,2022-03-05 04:39:00.000000,2022-03-05 11:41:00.000000,F501002
2,5016523961394,95.936600,2022-03-05 04:39:04.151000,R5010,10000411,896,2022-03-05 04:39:00.0000000 +01:00,2022-03-05 11:41:00.0000000 +01:00,2022-03-05 04:39:00.000000,2022-03-05 11:41:00.000000,H506003C
3,5016523980111,890.880000,2022-03-05 04:39:04.913000,R5010,10000411,896,2022-03-05 04:39:00.0000000 +01:00,2022-03-05 11:41:00.0000000 +01:00,2022-03-05 04:39:00.000000,2022-03-05 11:41:00.000000,P501000
4,5016523978452,0.705643,2022-03-05 04:39:05.000000,R5010,10000411,896,2022-03-05 04:39:00.0000000 +01:00,2022-03-05 11:41:00.0000000 +01:00,2022-03-05 04:39:00.000000,2022-03-05 11:41:00.000000,U501002H2
...,...,...,...,...,...,...,...,...,...,...,...
303145,5016523967137,29758.400000,2022-04-24 16:16:58.659000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000,U506000
303146,5016523956271,30.186604,2022-04-24 16:16:58.659000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000,U506001H2
303147,5016523955535,96.793000,2022-04-24 16:16:58.953000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000,F501097_MASS
303148,5016523952192,2715.840000,2022-04-24 16:16:59.021000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000,F501040C


In [39]:
new_ts_min = new_df['TimeStamp'].min()
new_ts_max = new_df['TimeStamp'].max()

print(new_ts_min, new_ts_max)

2022-03-05 04:39:02.294000 2022-04-24 16:16:59.318000


In [25]:
dropped =new_df.drop_duplicates(subset=['UniqueId', 'Value', 'TimeStamp', 'AssetUnitIdentifier',
       'Materialnumber', 'Batchnumber', 'SegmentStartTime', 'SegmentEndTime',
       'SegmentStartTime_formatted', 'SegmentEndTime_formatted'])

In [26]:
dropped.columns

Index(['UniqueId', 'Value', 'TimeStamp', 'AssetUnitIdentifier',
       'Materialnumber', 'Batchnumber', 'SegmentStartTime', 'SegmentEndTime',
       'SegmentStartTime_formatted', 'SegmentEndTime_formatted', 'BackendTag'],
      dtype='object')

In [27]:
dropped

,UniqueId,Value,TimeStamp,AssetUnitIdentifier,Materialnumber,Batchnumber,SegmentStartTime,SegmentEndTime,SegmentStartTime_formatted,SegmentEndTime_formatted,BackendTag
0,5016523970349,29.136250,2022-03-05 04:39:02.294000,R5010,10000411,896,2022-03-05 04:39:00.0000000 +01:00,2022-03-05 11:41:00.0000000 +01:00,2022-03-05 04:39:00.000000,2022-03-05 11:41:00.000000,S501000
1,5016523955926,277.568756,2022-03-05 04:39:03.760000,R5010,10000411,896,2022-03-05 04:39:00.0000000 +01:00,2022-03-05 11:41:00.0000000 +01:00,2022-03-05 04:39:00.000000,2022-03-05 11:41:00.000000,F501002
2,5016523961394,95.936600,2022-03-05 04:39:04.151000,R5010,10000411,896,2022-03-05 04:39:00.0000000 +01:00,2022-03-05 11:41:00.0000000 +01:00,2022-03-05 04:39:00.000000,2022-03-05 11:41:00.000000,H506003C
3,5016523980111,890.880000,2022-03-05 04:39:04.913000,R5010,10000411,896,2022-03-05 04:39:00.0000000 +01:00,2022-03-05 11:41:00.0000000 +01:00,2022-03-05 04:39:00.000000,2022-03-05 11:41:00.000000,P501000
4,5016523978452,0.705643,2022-03-05 04:39:05.000000,R5010,10000411,896,2022-03-05 04:39:00.0000000 +01:00,2022-03-05 11:41:00.0000000 +01:00,2022-03-05 04:39:00.000000,2022-03-05 11:41:00.000000,U501002H2
...,...,...,...,...,...,...,...,...,...,...,...
303145,5016523967137,29758.400000,2022-04-24 16:16:58.659000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000,U506000
303146,5016523956271,30.186604,2022-04-24 16:16:58.659000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000,U506001H2
303147,5016523955535,96.793000,2022-04-24 16:16:58.953000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000,F501097_MASS
303148,5016523952192,2715.840000,2022-04-24 16:16:59.021000,R5010,10000411,903,2022-04-24 11:28:00.0000000 +02:00,2022-04-24 16:17:00.0000000 +02:00,2022-04-24 11:28:00.000000,2022-04-24 16:17:00.000000,F501040C


In [ ]:
### # Pyspark Approach for Olgas H505 usecase

In [7]:
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/29 15:31:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/29 15:31:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [8]:
l4data = pd.DataFrame({'start_date':['2017-03-27 09:40:10','2017-01-10 22:10:11'],'end_date':['2017-04-20 10:30:22','2017-02-01 20:01:01']})
pimsdata = pd.DataFrame({'event_date':['2017-01-20 23:01:00','2017-01-27 09:40:10'],'value':[100,200]})

l4data['end_date'] = pd.to_datetime(l4data.end_date)
l4data['start_date'] = pd.to_datetime(l4data.start_date)
pimsdata['event_date'] = pd.to_datetime(pimsdata.event_date)

In [9]:
# reference: https://stackoverflow.com/questions/30630296/joining-two-spark-dataframes-on-time-timestamptype-in-python
l4data_sparkdf=spark.createDataFrame(l4data) 
pimsdata_sparkdf=spark.createDataFrame(pimsdata) 

In [10]:
joined_df = l4data_sparkdf.join(pimsdata_sparkdf) \
    .filter(l4data_sparkdf.start_date < pimsdata_sparkdf.event_date) \
    .filter(pimsdata_sparkdf.event_date < l4data_sparkdf.end_date)

In [11]:
joined_df.show()

+-------------------+-------------------+-------------------+-----+
|         start_date|           end_date|         event_date|value|
+-------------------+-------------------+-------------------+-----+
|2017-01-10 22:10:11|2017-02-01 20:01:01|2017-01-20 23:01:00|  100|
|2017-01-10 22:10:11|2017-02-01 20:01:01|2017-01-27 09:40:10|  200|
+-------------------+-------------------+-------------------+-----+



In [14]:
joined_df = l4data_sparkdf.join(pimsdata_sparkdf,(
    (l4data_sparkdf.start_date < pimsdata_sparkdf.event_date) 
    & (pimsdata_sparkdf.event_date < l4data_sparkdf.end_date)
))  
                                
                                

In [16]:
joined_df.show()

+-------------------+-------------------+-------------------+-----+
|         start_date|           end_date|         event_date|value|
+-------------------+-------------------+-------------------+-----+
|2017-01-10 22:10:11|2017-02-01 20:01:01|2017-01-20 23:01:00|  100|
|2017-01-10 22:10:11|2017-02-01 20:01:01|2017-01-27 09:40:10|  200|
+-------------------+-------------------+-------------------+-----+



In [ ]:
## Further Checks

# Time related functions that can be used in query..

# merge_asof
https://www.geeksforgeeks.org/pandas-merge_asof-function-in-python/
# Merge two Pandas DataFrames based on closest DateTime
https://www.geeksforgeeks.org/merge-two-pandas-dataframes-based-on-closest-datetime/
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.between_time.html